In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar
from scipy.integrate import simps


wavelength_range = [360,830]
def load_cmf(filename):
    data = pd.read_csv(filename, header=None, names=['wavelength', 'x', 'y', 'z'])
    cmf_wavelengths = data['wavelength'].values
    cmf_indices = np.where((cmf_wavelengths >= wavelength_range[0]) & (cmf_wavelengths <= wavelength_range[1]))[0]
    cmf_wavelengths = cmf_wavelengths[cmf_indices]
    x = np.interp(cmf_wavelengths, data['wavelength'].values, data['x'].values)
    y = np.interp(cmf_wavelengths, data['wavelength'].values, data['y'].values)
    z = np.interp(cmf_wavelengths, data['wavelength'].values, data['z'].values)
    cmf = {
        'wavelength': cmf_wavelengths,
        'x': x,
        'y': y,
        'z': z,
    }
    return cmf

def load_spd(filename, cmf):
    data = pd.read_csv(filename, sep='\t', skiprows=2, header=None, names=['wavelength', 'intensity'])
    spd_wavelengths = np.round(data['wavelength']).astype(int).values
    spd_indices = np.where((spd_wavelengths >= wavelength_range[0]) & (spd_wavelengths <= wavelength_range[1]))[0]
    spd_wavelengths = spd_wavelengths[spd_indices]
    spd_intensities = data['intensity'].values[spd_indices] 
    cmf_wavelengths = cmf['wavelength']
    x = np.interp(cmf_wavelengths, spd_wavelengths, spd_intensities)

    spd = {
        'wavelength': cmf_wavelengths,
        'intensity': x,
    }
    return spd

def spd_to_xyz(spd, wavelength, cmf):
    X = simps(spd * cmf['x'], wavelength)
    Y = simps(spd * cmf['y'], wavelength)
    Z = simps(spd * cmf['z'], wavelength)
    xyz = np.array([X, Y, Z])
    x = xyz[0] / np.sum(xyz)
    y = xyz[1] / np.sum(xyz)
    return np.array([x,y])

def color_gamut_point(device_model, cmf):
    point = []
    light = ['red', 'blue', 'darkgreen']
    for i in light:
        data_filename = f"{device_model}_{i}.txt"
        spd_data = load_spd(data_filename ,cmf)
        point.append(spd_to_xyz(spd_data['intensity'], spd_data['wavelength'], cmf))
    return np.array(point)

# P3 color gamut
P3_xy = np.array([
    [0.6800, 0.3200],
    [0.2650, 0.6900],
    [0.1500, 0.0600]
])

def calc_area_of_triangle(xy_points):
    A, B, C = xy_points
    return 0.5 * np.abs(np.cross(A - C, B - C))

# calculate the gamut
def calc_coverage(xy_device, P3_xy):
    area_divice = calc_area_of_triangle(xy_device)
    area_P3 = calc_area_of_triangle(P3_xy)
    color_coverage = area_divice/area_P3
    return color_coverage

In [6]:
### start calculation
cmf_data = load_cmf('CIE_cc_1931_2deg.csv')
phone = []
coverage = []
point = []

for i in range (12,15):
    dataset_filename = f"{i}_white.txt"
    # Load SPD data
    # spd_data = load_spd(dataset_filename,cmf_data)
    grmut_point = color_gamut_point (i, cmf_data)
    coverage_divice = calc_coverage(grmut_point, P3_xy)
    phone.append(f"iphone {i}")
    coverage.append(coverage_divice)
    point.append(color_gamut_point(i, cmf_data))

table = pd.DataFrame(data={"Phone": phone, "Color Coverage": coverage})
print(table)


       Phone  Color Coverage
0  iphone 12        0.370262
1  iphone 13        0.367885
2  iphone 14        0.332509
